In [267]:
import numpy as np
import pandas as pd
import pdfplumber
import tabula
import camelot
from datetime import datetime, timedelta
import os

In [268]:
# where to load the pdf file
_DATA_TABLEAU_PDF_PATH_ = "/Users/xingruchen/Dropbox/COV/Vaccine/data/tableau/pdf/"
# where to save the csv file 
# UTC time
_DATA_TABLEAU_CSV_PATH_U_ = "/Users/xingruchen/Dropbox/COV/Vaccine/data/tableau/csv_UTC/"
# EST time
_DATA_TABLEAU_CSV_PATH_E_ = "/Users/xingruchen/Dropbox/COV/Vaccine/data/tableau/csv_EST/"

In [269]:
# This is a local news-powered dashboard tracking vaccine dose allocations for 
# all 64 jurisdictions (50 states, 8 territories and 6 cities) and 5 federal agencies.
# The first Pfizer doses are expected to ship by December 15 (pending FDA approval), 
# followed by the first Moderna doses on December 22.
# Then, a second round of Pfizer doese will ship by the end of December, 
# which will be the second doses for the first group of individuals.

In [392]:
def pdf_to_csv(file):
    # get the update date and time
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    #text = page.extract_text()
    text = [item['text'] for item in page.extract_words()]
    update_date = text[text.index('Updated:') + 1]
    update_time = text[text.index('Updated:') + 2]
    update_time = update_time.replace('\ue353', ':')
    update_apm = text[text.index('Updated:') + 3]
    pdf.close()
    # Tableau time zone: UTC 
    # Universal Time Coordinated is 5 hours ahead of Eastern Standard Time
    update_datetime_UTC = datetime.strptime(update_date + ' ' + update_time + update_apm, '%m/%d/%Y %I:%M:%S%p')
    update_datetime_EST = update_datetime_UTC - timedelta(hours = 5)
    print(update_datetime_UTC, update_datetime_EST)
    # get the dataframe
    table = tabula.read_pdf(file, pages = 1, lattice = False, stream = True)
    df = table[0]
    df = df[df.columns[:-1]]
    # exceptions: (17)
    if len(df.columns) < 9:
        df_front = df[['State', 'Jurisdiction Type', 'Pfizer Shipment 1']]
        name_mid = df.columns[3]
        df_split_mid = pd.DataFrame(df[name_mid].str.split(' ', 1).tolist(),
                                columns = ['Pfizer Shipment 2','Pfizer Shipment 2 Affected'])
        name_back = df.columns[4]
        df_split_back = pd.DataFrame(df[name_back].str.split(' ', 3).tolist(),
                                         columns = ['Moderna Shipment 1','Moderna 2', 'Pfizer Shipment 3', 'Estimated December Doses'])

        df = pd.concat([df_front, df_split_mid, df_split_back], axis = 1, sort = False)    
    
    # exceptions: (17), (18), ...
    if 'Pfizer Shipment 2 Affected' in df.columns:
        columns_num = df.columns[2:].drop('Pfizer Shipment 2 Affected')  
    else:
        columns_num = df.columns[2:]
    
    df = df.replace('Null', np.nan)
    # numeric columns
    df[columns_num] = df[columns_num].apply(lambda x: x.str.replace(',', '').astype(float), axis = 1)
    #if ((df['Pfizer Shipment 2'] == df['Pfizer Shipment 2.1']) | (df['Pfizer Shipment 2'].isnull() & df['Pfizer Shipment 2.1'].isnull())).all():
    if 'Pfizer Shipment 2.1' in df.columns:
        df = df.drop(['Pfizer Shipment 2.1'], axis = 1)

    columns = {'State': 'state', 'Jurisdiction Type': 'jurisdiction', 'Total Population': 'population', 
               'Pfizer Shipment 1': 'Pfizer_1', 'Pfizer Shipment 2': 'Pfizer_2', 
               'Pfizer Shipment 2 Affected': 'Pfizer_2_affected', 'Pfizer Shipment 3': 'Pfizer_3',
               'Moderna Shipment 1': 'Moderna_1', 'Moderna 2': 'Moderna_2', 
               'Estimated December Doses': 'December_doses', 
               'Pfizer Shipment 1: Percent of Adult Population': 'Pfizer_1_adult_percent', 
               'December % of Adult Population': 'December_doses_adult_percent'}

    df = df.rename(columns = columns)

    df['update_time'] = update_datetime_UTC
    if 'Pfizer_2_affected' in df.columns:
        df = df[['update_time', 'state', 'jurisdiction', 'Pfizer_1', 'Pfizer_2', 'Pfizer_2_affected', 'Pfizer_3',
                'Moderna_1', 'Moderna_2', 'December_doses']] # 'population', 'Pfizer_1_adult_percent', 'December_doses_adult_percent'
    else:
        df = df[['update_time', 'state', 'jurisdiction', 'Pfizer_1', 'Pfizer_2', 'Pfizer_3',
                'Moderna_1', 'Moderna_2', 'December_doses']] # 'population', 'Pfizer_1_adult_percent', 'December_doses_adult_percent'
    
    # Path to the output CSV file that will be created. If the file already exists, it will be overwritten.
    df.to_csv(_DATA_TABLEAU_CSV_PATH_U_ + 'allocation_' + update_datetime_UTC.strftime("%Y%m%d%H%M%S") + '.csv', index = False)
    
    df['update_time'] = update_datetime_EST
    
    df.to_csv(_DATA_TABLEAU_CSV_PATH_E_ + 'allocation_' + update_datetime_EST.strftime("%Y%m%d%H%M%S") + '.csv', index = False)
    
    return update_datetime, df

In [393]:
file_list = os.listdir(_DATA_TABLEAU_PDF_PATH_)
file_list.sort()

In [403]:
file_list[1:-2]

['Allocation Table (0).pdf',
 'Allocation Table (1).pdf',
 'Allocation Table (10).pdf',
 'Allocation Table (11).pdf',
 'Allocation Table (12).pdf',
 'Allocation Table (13).pdf',
 'Allocation Table (14).pdf',
 'Allocation Table (15).pdf',
 'Allocation Table (16).pdf',
 'Allocation Table (17).pdf',
 'Allocation Table (18).pdf',
 'Allocation Table (19).pdf',
 'Allocation Table (2).pdf',
 'Allocation Table (20).pdf',
 'Allocation Table (3).pdf',
 'Allocation Table (4).pdf',
 'Allocation Table (5).pdf',
 'Allocation Table (6).pdf',
 'Allocation Table (7).pdf',
 'Allocation Table (8).pdf',
 'Allocation Table (9).pdf']

In [396]:
for i, name in enumerate(file_list[1:-2]):
    index = int(name.split(' ')[2][1:-5])
    if index > 20:
        file = _DATA_TABLEAU_PDF_PATH_ + name
        update_datetime, df = pdf_to_csv(file)
        #print(name)

2020-12-15 23:36:01 2020-12-15 18:36:01
2020-12-15 23:36:01 2020-12-15 18:36:01
2020-12-16 02:54:31 2020-12-15 21:54:31
2020-12-17 02:14:53 2020-12-16 21:14:53
2020-12-17 16:18:12 2020-12-17 11:18:12
2020-12-17 16:18:12 2020-12-17 11:18:12
2020-12-17 16:18:12 2020-12-17 11:18:12
2020-12-18 03:53:49 2020-12-17 22:53:49
2020-12-18 03:53:49 2020-12-17 22:53:49
2020-12-18 14:39:55 2020-12-18 09:39:55
2020-12-18 22:22:24 2020-12-18 17:22:24
2020-12-19 02:56:12 2020-12-18 21:56:12
2020-12-15 23:36:01 2020-12-15 18:36:01
2020-12-19 05:13:08 2020-12-19 00:13:08
2020-12-15 23:36:01 2020-12-15 18:36:01
2020-12-15 23:36:01 2020-12-15 18:36:01
2020-12-15 23:36:01 2020-12-15 18:36:01
2020-12-16 01:36:07 2020-12-15 20:36:07
2020-12-16 01:36:07 2020-12-15 20:36:07
2020-12-16 02:54:31 2020-12-15 21:54:31
2020-12-16 02:54:31 2020-12-15 21:54:31


In [390]:
table = tabula.read_pdf(_DATA_TABLEAU_PDF_PATH_ + 'Allocation Table (18).pdf', pages = 1, lattice = False, stream = True)
df = table[0]
df = df[df.columns[:-1]]